In [46]:
#https://figshare.com/articles/smile_annotations_final_csv/3187909/2

In [47]:
#Data Analysis and Preprocessing

In [48]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [49]:
df= pd.read_csv(
    'Data/smileannotationsfinal.csv',
    names = ['id','text','category']
    )
df.set_index('id',inplace=True)

In [50]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [51]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [52]:
df = df[~df.category.str.contains('\|')]

In [53]:
df = df[df.category != 'nocode']

In [54]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [55]:
possible_labels = df.category.unique()

In [56]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [57]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [58]:
#Train Test Split

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values, #X
    df.label.values, #y
    test_size=0.15,
    random_state=17,
    stratify=df.label.values #ensure all categories are trained since there is uneven distribution
)

In [61]:
y_train

array([0, 0, 0, ..., 0, 0, 2])

In [62]:
df['data_type'] = ['not_set']*df.shape[0]

In [63]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [64]:
df.loc[X_train,'data_type'] = 'train'
df.loc[X_val,'data_type'] = 'val'

In [65]:
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

In [92]:
import numpy as np

sent = [tokenizer.tokenize(i) for i in df.text.values]
print(sent[0:2])
print(len(sent))

[['dorian', 'gray', 'with', 'rainbow', 'scarf', '#', 'love', '##win', '##s', '(', 'from', '@', 'british', '##mus', '##eum', 'http', ':', '/', '/', 't', '.', 'co', '/', 'q', '##4', '##x', '##sw', '##l', '##0', '##es', '##u', ')', 'http', ':', '/', '/', 't', '.', 'co', '/', 'h', '##0', '##ev', '##bt', '##b', '##wr', '##q'], ['@', 'selects', '##how', '##case', '@', 'tate', '_', 'st', '##ives', '.', '.', '.', 'replace', 'with', 'your', 'wish', 'which', 'the', 'artist', 'uses', 'in', 'next', 'installation', '!', 'it', 'was', 'en', '##tral', '##ling', '!']]
1481


In [94]:
wordcounts = [len(s) for s in sent]
print(wordcounts[0:5])
print(max(wordcounts))

[47, 30, 31, 23, 16]
103


In [66]:
#Bert Tokenizer

In [67]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [68]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',  #use pretrained bert tokenizer
    do_lower_case=True  #turn sentences into lower case
)

In [71]:
encoded_data_train = tokenizer.batch_encode_plus( #iterates over sentences
    df[df.data_type=='train'].text.values,  #get the sentences for training
    add_special_tokens=True,
    return_attention_mask=True, #returns attention masks 1,0 so that it knows what to focus on e.g. not focus padding
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(  #iterates over sentences
    df[df.data_type=='val'].text.values,  #get the sentences for training
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

#inputs for BERT
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [76]:
type(encoded_data_train)

transformers.tokenization_utils_base.BatchEncoding

In [27]:
#tokenizer.batch_encode_plus(df[df.data_type=='train'].text.values) returns array of input_id arrays
#similar to tokenizer._call_ ???

In [24]:
dataset_train = TensorDataset(input_ids_train,
                             attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val,
                             attention_masks_val, labels_val)

In [25]:
len(dataset_train)

1258

In [26]:
len(dataset_val)

223

In [27]:
#Set up BERT pretrained

In [28]:
from transformers import BertForSequenceClassification

In [29]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [30]:
#Create data loaders helps iterate through dataset batches nicely

In [31]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [32]:
batch_size = 4 #32 #not to overload memory

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

In [33]:
#Setting Up Optimizer

In [34]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [35]:
#find weights
optimizer = AdamW(
    model.parameters(),
    lr=1e-5, #learning rate should be approx. 2e-5 > 5e-5
    eps=1e-8 #epsilon
)

In [36]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs #total training steps = data to train*epochs
)

In [37]:
#Define performance metrics

In [38]:
import numpy as np

In [39]:
from sklearn.metrics import f1_score

In [40]:
def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat,preds_flat,average='weighted')

In [41]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class:{label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [42]:
#create training loop

In [43]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #cuda if gpu is available
model.to(device)

print(device)

cpu


In [45]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs) #**inputs flattens dictionary into inputs
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()  #incase want to use gpu??
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1,epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    
    #to show progress by tqdm
    progress_bar = tqdm(dataloader_train,
                       desc = 'Epoch {:1d}'.format(epoch),
                       leave=False, #will replace tqdm when done
                        disable=False
                       )
    
    for batch in progress_bar:
        
        model.zero_grad() #start with 0 gradient
        
        batch = tuple(b.to(device) for b in batch)
        
        #inputs into model
        inputs = {
            'input_ids':batch[0],
            'attention_mask':batch[1],
            'labels':batch[2]
        }
        
        outputs = model(**inputs)
        
        loss=outputs[0]
        loss_train_total += loss.item()
        loss.backward() #backward propagation??
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        #help with the calculating gradient by clipping
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
    
    #displays
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg} ')
    
    #for validation set
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions,true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')


Epoch {epoch}
Training loss: 0.7418815144352496 



Validation loss: 0.6122817354542869
F1 Score (weighted): 0.7764636930411201


In [ ]:
#Loading and Evaluating Model

In [47]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [49]:
model.to(device) #select the device
pass

In [50]:
model.load_state_dict(
    torch.load('Models/BERT_ft_epoch10.model',
              map_location=torch.device('cpu')))

<All keys matched successfully>

In [51]:
_,predictions,true_vals = evaluate(dataloader_val)

In [52]:
accuracy_per_class(predictions, true_vals)

Class:happy
Accuracy: 163/171

Class:not-relevant
Accuracy: 19/32

Class:angry
Accuracy: 7/9

Class:disgust
Accuracy: 0/1

Class:sad
Accuracy: 2/5

Class:surprise
Accuracy: 2/5



In [ ]:
#Google Colab --GPU Instance
#batch_size = 32
#epoch =10